In [ ]:
import pickle
from scipy.sparse import coo_matrix
import json

import pickle
from scipy.sparse import coo_matrix
import glob

# originalMultiLabelData
for x in glob.glob("data/snippets_data/*"):

    with open(x, 'rb') as fp:
        if 'pkl' in x:
            adj = pickle.load(fp)
            print(x)
            print(type(adj))         # <class 'scipy.sparse.coo.coo_matrix'>
            print(adj.shape)         # (예: (1000, 500))
            # print(adj.nnz)           # non-zero 원소 수
            
        else:
            print(x)
            x = json.load(fp)
            print(len(x))
            
glob.glob("data/snippets_data/*")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data/jail_breaker_data/dataset_pairs.tsv", sep='\t')

In [ ]:
clean_labels = []

for label in df.label.unique():
    if label.startswith('TemplateJailbreak'):
        clean_labels.append('TemplateJailbreak')
    else:
        clean_labels.append(label)

clean_labels = np.array(clean_labels)
unique_labels = np.unique(clean_labels)
label2idx = {label: idx for idx, label in enumerate(unique_labels)}

# 정수 인덱스 라벨로 변환
numeric_labels = [label2idx[label] for label in clean_labels]

In [ ]:
numeric_pair = dict([(str(k), v) for k, v in zip(clean_labels, numeric_labels)])

In [ ]:
df['numeric_label'] = df.label.apply(lambda x: numeric_pair['TemplateJailbreak'] if x.startswith('TemplateJailbreak') else numeric_pair[x])

In [ ]:
print(df.head())

In [ ]:
docs = list(df.text)

docs

In [ ]:
# with open('data/jail_breaker/labels.json', 'w') as fp:
#     json.dump(list(df.numeric_label), fp)

##### Entity Build

In [ ]:
import spacy
from scipy.sparse import coo_matrix
import pickle
import numpy as np
import os

# 1. spaCy 모델 로딩
nlp = spacy.load("en_core_web_sm")

# 2. 문서 리스트
# docs = [...]

# 3. 문서별 엔티티 추출
doc_entities = []
for text in docs:
    doc = nlp(text)
    ents = set(ent.text.strip().lower() for ent in doc.ents if ent.text.strip())
    doc_entities.append(ents)

# 4. 전체 엔티티 목록 만들기
all_ents = sorted(set(e for ents in doc_entities for e in ents))
ent2idx = {ent: idx for idx, ent in enumerate(all_ents)}

# 5. 희소 행렬 만들기
rows, cols, data = [], [], []
for doc_idx, ents in enumerate(doc_entities):
    for ent in ents:
        rows.append(doc_idx)
        cols.append(ent2idx[ent])
        data.append(1)

A_qe = coo_matrix((data, (rows, cols)), shape=(len(docs), len(all_ents)))

# 6. 희소행렬 저장
os.makedirs("data/jail_breaker_data", exist_ok=True)
with open("data/jail_breaker_data/adj_query2entity.pkl", "wb") as f:
    pickle.dump(A_qe, f)

# 7. 엔티티 임베딩 바로 생성 및 저장
dim = 300
np.random.seed(42)
entity_emb = np.random.randn(len(all_ents), dim).astype(np.float32)
with open("data/jail_breaker_data/entity_emb.pkl", "wb") as f:
    pickle.dump(entity_emb, f)

print(f"adj_query2entity shape: {A_qe.shape}")
print(f"entity_emb shape: {entity_emb.shape}")

In [ ]:
# import spacy
# from scipy.sparse import coo_matrix
# import pickle

# # 1. spaCy 모델 로딩
# nlp = spacy.load("en_core_web_sm")

# # 2. 문서 리스트
# docs = docs

# # 3. 문서별 엔티티 추출
# doc_entities = []
# for text in docs:
#     doc = nlp(text)
#     ents = set(ent.text.strip().lower() for ent in doc.ents if ent.text.strip())
#     doc_entities.append(ents)

# # 4. 전체 엔티티 목록 만들기
# all_ents = sorted(set(e for ents in doc_entities for e in ents))
# ent2idx = {ent: idx for idx, ent in enumerate(all_ents)}

# # 5. 희소 행렬 만들기
# rows, cols, data = [], [], []
# for doc_idx, ents in enumerate(doc_entities):
#     for ent in ents:
#         rows.append(doc_idx)
#         cols.append(ent2idx[ent])
#         data.append(1)

# A_qe = coo_matrix((data, (rows, cols)), shape=(len(docs), len(all_ents)))

# # 6. 저장
# with open("data/jail_breaker/adj_query2entity.pkl", "wb") as f:
#     pickle.dump(A_qe, f)

##### Word Build

In [ ]:
import spacy
from scipy.sparse import coo_matrix
import pickle

# 1. spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# 2. 문서 리스트 (이미 정의된 상태라고 가정)
# docs = [...]

# 3. 문서별 단어 토큰 추출
doc_words = []
for text in docs:
    doc = nlp(text)
    words = set(token.text.lower() for token in doc if token.is_alpha and not token.is_stop)
    doc_words.append(words)

# 4. 전체 단어 목록 및 인덱싱
all_words = sorted(set(w for words in doc_words for w in words))
word2idx = {word: idx for idx, word in enumerate(all_words)}

# 5. 희소 행렬 구성
rows, cols, data = [], [], []
for doc_idx, words in enumerate(doc_words):
    for word in words:
        rows.append(doc_idx)
        cols.append(word2idx[word])
        data.append(1)

A_qw = coo_matrix((data, (rows, cols)), shape=(len(docs), len(all_words)))

# 6. 저장
with open("data/jail_breaker_data/adj_query2word.pkl", "wb") as f:
    pickle.dump(A_qw, f)

In [ ]:
import spacy
import numpy as np
import pickle
import os

# 1. spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# 2. docs에서 단어 집합 추출 (중복 제거)
words = set()
for text in docs:
    doc = nlp(text)
    for token in doc:
        if token.is_alpha and not token.is_stop:
            words.add(token.text.lower())
words = sorted(words)

# 3. 임베딩 차원
dim = 300

# 4. 랜덤 임베딩 생성
np.random.seed(42)
word_emb = np.random.randn(len(words), dim).astype(np.float32)

# 5. 저장
with open("data/jail_breaker_data/word_emb.pkl", "wb") as f:
    pickle.dump(word_emb, f)

print(f"word_emb shape: {word_emb.shape}")

#### Tag Build

In [ ]:
import spacy
from scipy.sparse import coo_matrix
import pickle

# 1. spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# 2. 문서 리스트
# docs = [...]

# 3. 문서별 POS 태그 추출
doc_tags = []
for text in docs:
    doc = nlp(text)
    tags = set(token.pos_ for token in doc if token.is_alpha)
    doc_tags.append(tags)

# 4. 전체 태그 인덱싱
all_tags = sorted(set(t for tags in doc_tags for t in tags))
tag2idx = {tag: idx for idx, tag in enumerate(all_tags)}

# 5. 희소행렬 구성
rows, cols, data = [], [], []
for doc_idx, tags in enumerate(doc_tags):
    for tag in tags:
        rows.append(doc_idx)
        cols.append(tag2idx[tag])
        data.append(1)

A_qt = coo_matrix((data, (rows, cols)), shape=(len(docs), len(all_tags)))

# 6. 저장
with open("data/jail_breaker/adj_query2tag.pkl", "wb") as f:
    pickle.dump(A_qt, f)

In [ ]:
import spacy
from scipy.sparse import coo_matrix
import pickle
from itertools import combinations
import os

# 1. spaCy 모델 로딩
nlp = spacy.load("en_core_web_sm")

# 2. 문서 리스트
# docs = [...]

# 3. 문서별 POS 태그 수집
doc_tags = []
for text in docs:
    doc = nlp(text)
    tags = set(token.pos_ for token in doc if token.is_alpha)
    doc_tags.append(tags)

# 4. 태그 인덱스화
all_tags = sorted(set(t for tags in doc_tags for t in tags))
tag2idx = {tag: idx for idx, tag in enumerate(all_tags)}

# 5. 태그–태그 연결 (동일 문서에 등장한 쌍)
edges = set()
for tags in doc_tags:
    idxs = [tag2idx[t] for t in tags]
    for i, j in combinations(idxs, 2):
        a, b = sorted((i, j))
        edges.add((a, b))

# 6. 행렬 구성
rows, cols, data = [], [], []
for i, j in edges:
    rows.append(i)
    cols.append(j)
    data.append(1)
    rows.append(j)
    cols.append(i)
    data.append(1)

A_tag = coo_matrix((data, (rows, cols)), shape=(len(all_tags), len(all_tags)))

# 7. 저장
# os.makedirs("data/jail_breaker", exist_ok=True)

with open("data/jail_breaker/adj_tag.pkl", "wb") as f:
    pickle.dump(A_tag, f)

In [ ]:
# import spacy
# import numpy as np
# import pickle
# import os

# # 1. spaCy 모델 로드
# nlp = spacy.load("en_core_web_sm")

# # 2. docs에서 단어 집합 추출 (중복 제거)
# words = set()
# for text in docs:
#     doc = nlp(text)
#     for token in doc:
#         if token.is_alpha and not token.is_stop:
#             words.add(token.text.lower())
# words = sorted(words)

# # 3. 임베딩 차원
# dim = 300

# # 4. 랜덤 임베딩 생성
# np.random.seed(42)
# word_emb = np.random.randn(len(words), dim).astype(np.float32)

# # 5. 저장
# os.makedirs("data/jail_breaker", exist_ok=True)
# with open("data/jail_breaker/word_emb.pkl", "wb") as f:
#     pickle.dump(word_emb, f)

# print(f"word_emb shape: {word_emb.shape}")

##### Split Train-Test

In [ ]:
import json
from sklearn.model_selection import train_test_split
import os

# 전체 문서 수
num_docs = len(docs)

# 전체 인덱스
all_indices = list(range(num_docs))

# train/test 분할 (ex: 80/20)
train_idx, test_idx = train_test_split(all_indices, test_size=0.2, random_state=42)

# 저장 경로
os.makedirs("data/jail_breaker", exist_ok=True)

# 저장
with open("data/jail_breaker/train_idx.json", "w") as f:
    json.dump(train_idx, f)

with open("data/jail_breaker/test_idx.json", "w") as f:
    json.dump(test_idx, f)

##### Data Check

In [ ]:
# Data Shape Check
for x in glob.glob('data/jail_breaker_data/*'):
    with open(x, 'rb') as fp:
        if 'pkl' in x:
            adj = pickle.load(fp)
            print(x)
            print(type(adj))         # <class 'scipy.sparse.coo.coo_matrix'>
            print(adj.shape)         # (예: (1000, 500))
            # print(adj.nnz)           # non-zero 원소 수
            
        elif 'json' in x:
            print(x)
            x = json.load(fp)
            print(len(x))

In [41]:
with open('/home/bighillanal/SimSTCFork/train_log.md', 'w') as fp:
    fp.close()

!python train.py --dataset jail_breaker

cuda:0
jail_breaker
True
10000
231
211
9558
data process time: 2.0872080326080322
Epoch 1, train_loss: 13.0739, train_acc: 0.0866, time: 0.5890

Epoch 2, train_loss: 12.7130, train_acc: 0.1861, time: 0.2243

Epoch 3, train_loss: 12.5218, train_acc: 0.1558, time: 0.2233

Epoch 4, train_loss: 12.3863, train_acc: 0.2424, time: 0.2233

Epoch 5, train_loss: 12.2864, train_acc: 0.2771, time: 0.2237

Valid  loss: 2.2574  acc: 0.3270  f1: 0.2725
Test  loss: 2.3658 acc: 0.3001 f1: 0.1849 time: 0.0759
Epoch 6, train_loss: 12.2712, train_acc: 0.2944, time: 0.2233

Epoch 7, train_loss: 12.2765, train_acc: 0.3117, time: 0.2231

Epoch 8, train_loss: 12.2251, train_acc: 0.3593, time: 0.2232

Epoch 9, train_loss: 12.2202, train_acc: 0.3723, time: 0.2234

Epoch 10, train_loss: 12.2220, train_acc: 0.3420, time: 0.2233

Valid  loss: 2.2221  acc: 0.3412  f1: 0.2910
Test  loss: 2.2848 acc: 0.3661 f1: 0.2208 time: 0.0756
Epoch 11, train_loss: 12.1807, train_acc: 0.3420, time: 0.2232

Epoch 12, train_loss: 1

In [ ]:
!nvidia-smi